In [8]:
import numpy as np
import scipy
import matplotlib.pyplot as plt



SAMPLEFREQ = 44100
FRAMESIZE = 1024
NOFFRAMES = 220

## Record Audio

In [9]:
%%script false

import pyaudio

def recordAudio():
    p = pyaudio.PyAudio()
    FORMAT = pyaudio.paInt16

    stream = p.open(format=FORMAT, channels=1, rate=SAMPLEFREQ, input=True, frames_per_buffer=FRAMESIZE)
    data = stream.read(NOFFRAMES*FRAMESIZE)
    decoded = np.frombuffer(data, "int16")
    stream.stop_stream()
    stream.close()
    return decoded

data = recordAudio()
plt.plot(data)
plt.ylabel("Amplitude")
plt.xlabel("Frames")
plt.show()



CalledProcessError: Command 'b'\nimport pyaudio\n\ndef recordAudio():\n    p = pyaudio.PyAudio()\n    FORMAT = pyaudio.paInt16\n\n    stream = p.open(format=FORMAT, channels=1, rate=SAMPLEFREQ, input=True, frames_per_buffer=FRAMESIZE)\n    data = stream.read(NOFFRAMES*FRAMESIZE)\n    decoded = np.frombuffer(data, "int16")\n    stream.stop_stream()\n    stream.close()\n    return decoded\n\ndata = recordAudio()\nplt.plot(data)\nplt.ylabel("Amplitude")\nplt.xlabel("Frames")\nplt.show()\n\n\n'' returned non-zero exit status 1.

# Input normalisieren

In [ ]:
def normInput(decoded):
    for i, v in enumerate(decoded):
        if v > 500:
            decoded = decoded[i:]
            break
    if len(decoded) > SAMPLEFREQ:
        decoded = decoded[:SAMPLEFREQ]
    while len(decoded) < SAMPLEFREQ:
        np.append(decoded, 0)
    return decoded

plt.plot(normInput(data))
plt.ylabel("Amplitude")
plt.xlabel("Frames")
plt.show()

In [ ]:
%%script false

typ = "TestDaten"
befehl = ""
name = "hannes"
for i in range(5):
    print("Start")
    data = normInput(recordAudio())
    if len(name) == 0:
        np.save(f"Messdaten/{typ}/{befehl}/{befehl}{i}", data)
    else:
        np.save(f"Messdaten/{typ}/{name}/{befehl}/{befehl}{i}", data)
    plt.plot(data)
    plt.show()

# Import npy file

In [ ]:
data = np.load("Messdaten/Referenzen/hoch/hoch0.npy")

# Amplitudenspektrum

In [ ]:
fourier_transform = np.fft.fft(data)
amplituden = np.abs(fourier_transform)

plt.plot(amplituden[:1000])
plt.xlabel('Frames')
plt.ylabel('Amplitude')
plt.title('Amplitude Spectrum')

# Windowing

In [ ]:
def windowing(win_data):
    windowSize = 512
    window = scipy.signal.windows.gaussian(windowSize, std=windowSize/4)
    windows_fft = []

    for i in range(0, SAMPLEFREQ, int(windowSize/2)):
        localData = data[i:i+windowSize]
        if len(localData) != 512:
            continue
        local_fft = np.fft.fft(localData*window)
        windows_fft.append(local_fft)
    mean_fft = np.array(windows_fft).mean(axis=0)
    amplituden = np.abs(mean_fft)
    return amplituden

In [ ]:
plt.plot(windowing(data)[:100])
plt.title("Amplitude Spectrum")
plt.xlabel("Amplitude")
plt.ylabel("Frequenz (hz)")
plt.show()

# Daten einlesen

In [ ]:

befehle = ["hoch", "runter", "links", "rechts"]

ref_dict = {}
for idx_befehl, befehl in enumerate(befehle):
    ref_dict[befehl] = []
    for i in range(5):
        data = np.load(f"Messdaten/Referenzen/{befehl}/{befehl}{i}.npy")
        ref_dict[befehl].append(windowing(data))
    ref_dict[befehl] = np.array(ref_dict[befehl]).mean(axis=0)
    plt.subplot(2, 2, idx_befehl + 1)
    plt.plot(ref_dict[befehl][:20])
    plt.title(befehl)

plt.show()


# Testdatensätze

In [ ]:
tests_dict = {}
namen = ["jonas", "hannes"]

for idx_name, name in enumerate(namen):
    tests_dict[name] = {}
    for idx_befehl, befehl in enumerate(befehle):
        tests_dict[name][befehl] = []
        for i in range(5):
            data = np.load(f"Messdaten/TestDaten/{name}/{befehl}/{befehl}{i}.npy")
            tests_dict[name][befehl].append(windowing(data))
        tests_dict[name][befehl] = np.array(tests_dict[name][befehl]).mean(axis=0)
        plt.subplot(3, 4, idx_name*4 + idx_befehl + 1)
        plt.plot(tests_dict[name][befehl][:20])
        plt.title(f"{name}: {befehl}")

plt.show()


# Korrelation

In [ ]:
def std_ab(d):
    mean = np.mean(d)
    return np.sqrt(sum(np.power(x - mean, 2) for x in d) / len(d))

def get_korr(input, ref):
    xMean = np.mean(input)
    yMean = np.mean(ref)
    length = len(input)
    kovarianz = sum((x - xMean) * (y - yMean)for x, y in zip(input, ref)) / length
    return kovarianz / (std_ab(input) * std_ab(ref))



# Vergleich

In [ ]:
korr_dict = {}
for befehl in befehle:
    korr_dict[befehl] = {}
    ref = ref_dict[befehl]
    for name in namen:
        korr_dict[befehl][name] = get_korr(tests_dict[name][befehl], ref)


for befehl, names in korr_dict.items():
    for name, korr in names.items():
        print(f"{name} mit Befehlt {befehl} korreliert mit {korr}")



## Testdaten verglichen mit allen Referenzen

In [ ]:
# befehl_dict {"hoch", daten, "rechts", daten, usw.}
for name, befehl_dict in tests_dict.items():
    score = 0
    print(f"{name.title()}:")

    for befehl, testDaten in befehl_dict.items():
        korrs = []
        # ref_befehl = welcher befehl (zb. "hoch"), ref = refDaten
        for ref_befehl, ref in ref_dict.items():

            korrs.append((ref_befehl, get_korr(testDaten, ref)))
        korrs.sort(key=lambda t: t[1], reverse=True)
        print(f"'{befehl.title()}' Korreliert am besten mit '{korrs[0][0].title()}': {korrs[0][1]}")
        if befehl == korrs[0][0]:
            score += 1
    print(f"Score: {(score / 4) * 100}%")
